In [1]:
import numpy as np

In [2]:
def get_sequences(p, n):
    seqs = set()
    row = np.array([1], dtype=np.int64)
    
    ub = 1
    while ub <= n:
        ub *= p
    ub *= p * p
    
    for _ in range(ub):
        for i in range(len(row) - n + 1):
            seqs.add(tuple(row[i : i + n]))

        row = np.concatenate([row, [0]])
        row[1:] += row[:len(row) - 1]
        row %= p
        
    return seqs

len(get_sequences(p=5, n=3))

101

In [3]:
get_sequences(p=3, n=3)

{(0, 0, 0),
 (0, 0, 1),
 (0, 0, 2),
 (0, 1, 0),
 (0, 1, 1),
 (0, 1, 2),
 (0, 2, 0),
 (0, 2, 1),
 (0, 2, 2),
 (1, 0, 0),
 (1, 0, 1),
 (1, 0, 2),
 (1, 1, 0),
 (1, 1, 2),
 (1, 2, 0),
 (1, 2, 1),
 (1, 2, 2),
 (2, 0, 0),
 (2, 0, 1),
 (2, 0, 2),
 (2, 1, 0),
 (2, 1, 1),
 (2, 1, 2),
 (2, 2, 0),
 (2, 2, 1)}